# Deep Nets with TF Abstractions

Let's explore a few of the various abstractions that TensorFlow offers. You can check out the tf.contrib documentation for more options.

# The Data

In [2]:
from sklearn.datasets import load_iris
iris_data = load_iris()
print(type(iris_data))

<class 'sklearn.utils.Bunch'>


The data is a sklearn.utils.Bunch object, which is very similar to a dictionary.

In [3]:
iris_data.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename'])

You can get a full description with **print(iris_data.DESCR)**. For now, let's go ahead and grab the features and the labels for the data.

In [4]:
feat_data = iris_data['data']
labels = iris_data['target']

In [14]:
feat_data[50]

array([7. , 3.2, 4.7, 1.4])

In [5]:
labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [10]:
iris_data['target_names']

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

### Train Test Split

As with any machine learning model, you should do some sort of test train split so you can evaluate your model's performance. Because this particular dataset is small, we'll just do a simple 70/30 train test split and we won't have any holdout data set.

Again, we'll use SciKit-Learn here for convienence:

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(feat_data,
                                                    labels,
                                                    test_size=0.3,
                                                   random_state=101)

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


### Scale the Data

With Neural Network models, its important to scale the data, again we can do this easily with SciKit Learn (I promise we'll get to TensorFlow soon!)

In [15]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

Keep in mind we only fit the scaler to the training data, we don't want to assume we'll have knowledge of future test data. 

In [16]:

scaled_x_train = scaler.fit_transform(X_train)
scaled_x_test = scaler.transform(X_test)

# Abstractions

With our data set up, its now time to explore some TensorFlow abstractions! Let's start with the Estimator API, its one the abstractions featured in the official documentation tutorials.

## Estimator API

We first start by importing both tensorflow and the estimator API.

In [17]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow.compat.v1 import estimator 

Instructions for updating:
non-resource variables are not supported in the long term


The estimator API can perform both Deep Neural Network Classification and Regression, as well as straight Linear Classification and Linear Regression. You can  

In [18]:
X_train.shape

(105, 4)

In [19]:
feat_cols = [tf.feature_column.numeric_column("x", shape=[4])]

In [20]:
deep_model = estimator.DNNClassifier(hidden_units=[20,20,20],
            feature_columns=feat_cols,
            n_classes=3,
            optimizer=tf.train.GradientDescentOptimizer(
                learning_rate=0.01) )

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Farid\\AppData\\Local\\Temp\\tmprca9ac_5', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [22]:
input_fn = estimator.inputs.numpy_input_fn(
    x={'x':scaled_x_train},
    y=y_train,
    shuffle=True,
    batch_size=10,
    num_epochs=5)

In [23]:
deep_model.train(input_fn=input_fn,steps=500)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\Farid\AppData\Local\Temp\tmprca9ac_5\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 10.460897, step = 1
INFO:tensorflow:Calling checkpoint listeners before saving checkpoi

In [24]:
input_fn_eval = estimator.inputs.numpy_input_fn(
    x={'x':scaled_x_test},
    shuffle=False)

In [25]:
preds = list(deep_model.predict(input_fn=input_fn_eval))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Farid\AppData\Local\Temp\tmprca9ac_5\model.ckpt-53
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [27]:
preds[0]

{'logits': array([ 2.2241108, -1.9611747, -2.9322546], dtype=float32),
 'probabilities': array([0.97945064, 0.01490514, 0.00564419], dtype=float32),
 'class_ids': array([0], dtype=int64),
 'classes': array([b'0'], dtype=object),
 'all_class_ids': array([0, 1, 2]),
 'all_classes': array([b'0', b'1', b'2'], dtype=object)}

In [28]:
predictions = [p['class_ids'][0] for p in preds]

In [29]:
from sklearn.metrics import confusion_matrix,classification_report

In [30]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       0.00      0.00      0.00        20
           2       0.38      1.00      0.55        12

    accuracy                           0.56        45
   macro avg       0.46      0.67      0.52        45
weighted avg       0.39      0.56      0.43        45



C:\Users\Farid\anaconda3-NEW\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


____________
______________

# TensorFlow Keras

### Create the Model

In [31]:
from tensorflow.compat.v1.keras import models

In [32]:
dnn_keras_model = models.Sequential()

### Add Layers to the model

In [33]:
from tensorflow.compat.v1.keras import layers

In [34]:
dnn_keras_model.add(
    layers.Dense(
        units=20,
        input_dim=4,
        activation='relu'))

In [35]:
dnn_keras_model.add(layers.Dense(
    units=20,
    activation='relu'))
dnn_keras_model.add(
    layers.Dense(units=20,
                 activation='relu'))

In [36]:
dnn_keras_model.add(
    layers.Dense(units=3,
                 activation='softmax'))

### Compile the Model

In [37]:
from tensorflow.compat.v1.keras import losses,optimizers,metrics

In [ ]:
# explore these
# losses.

In [ ]:
#optimizers.

In [38]:
losses.sparse_categorical_crossentropy

<function keras.losses.sparse_categorical_crossentropy(y_true, y_pred, from_logits=False, axis=-1)>

In [39]:
optimizers.Adam

keras.optimizer_v2.adam.Adam

In [40]:
dnn_keras_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

### Train Model

In [41]:
dnn_keras_model.fit(scaled_x_train,
                    y_train,
                    epochs=50)

Train on 105 samples
Epoch 1/50
105/105 [==============================] - 0s 2ms/sample - loss: 1.0960 - acc: 0.6190
Epoch 2/50
105/105 [==============================] - 0s 162us/sample - loss: 1.0729 - acc: 0.6190
Epoch 3/50
105/105 [==============================] - 0s 114us/sample - loss: 1.0497 - acc: 0.6381
Epoch 4/50
105/105 [==============================] - 0s 181us/sample - loss: 1.0271 - acc: 0.6381
Epoch 5/50
105/105 [==============================] - 0s 219us/sample - loss: 1.0074 - acc: 0.6381
Epoch 6/50
105/105 [==============================] - 0s 162us/sample - loss: 0.9897 - acc: 0.6381
Epoch 7/50
105/105 [==============================] - 0s 124us/sample - loss: 0.9722 - acc: 0.6381
Epoch 8/50
105/105 [==============================] - 0s 219us/sample - loss: 0.9537 - acc: 0.6286
Epoch 9/50
105/105 [==============================] - 0s 124us/sample - loss: 0.9335 - acc: 0.6095
Epoch 10/50
105/105 [==============================] - 0s 238us/sample - loss: 0.9125 - ac

In [42]:
import numpy as np

In [43]:
predictions = dnn_keras_model.predict(scaled_x_test)
classes=np.argmax(predictions,axis=1)

C:\Users\Farid\anaconda3-NEW\lib\site-packages\keras\engine\training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


In [44]:
print(classification_report(classes,y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       0.75      1.00      0.86        15
           2       1.00      0.71      0.83        17

    accuracy                           0.89        45
   macro avg       0.92      0.90      0.89        45
weighted avg       0.92      0.89      0.89        45

